In [51]:

import osmnx as ox
import folium
import pandas as pd
import numpy as np
import utils
import folium_utils
import networkx as nx

# from IPython.display import display
# pd.set_option('display.max_rows', None)
                                
latitude_start = 35.410138
longitude_start = 136.964248
latitude_end = 35.228897
longitude_end = 137.426587

# キャッシュを使う
ox.config(use_cache=True, log_console=True)

graph = ox.graph_from_bbox(north=max(latitude_start, latitude_end),
                           south=min(latitude_start, latitude_end),
                           east=max(longitude_start, longitude_end),
                           west=min(longitude_start, longitude_end),
                           network_type='drive',
                           simplify=True,
                           retain_all=True,
                           custom_filter='["highway"~"tertiary"]')

# グラフデータをGeoDataFrameに変換  
gdf_nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False)
gdf_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)

# 3座標間の角度を求める
def calculate_angle_between_vectors(A, B, C):
    vector_AB = np.array(B) - np.array(A)
    vector_BC = np.array(C) - np.array(B)
    
    dot_product = np.dot(vector_AB, vector_BC)
    norm_AB = np.linalg.norm(vector_AB)
    norm_BC = np.linalg.norm(vector_BC)
    
    cosine_theta = dot_product / (norm_AB * norm_BC)
    angle_rad = np.arccos(cosine_theta)
    angle_deg = np.degrees(angle_rad)
    return angle_deg

# 座標間の角度の変化の合計値を求める
gdf_edges['geometory_angle_total'] = gdf_edges['geometry'].apply(
    lambda x: sum([calculate_angle_between_vectors(x.coords[i-1], x.coords[i], x.coords[i+1]) for i in range(1, len(x.coords)-1)])
)

# エッジの距離が1000m以上かつ、角度の変化が120度以上のエッジが峠の候補とする
lower_bound_meter = 1000
gdf_edges['is_target'] = np.where((gdf_edges['length'] >= lower_bound_meter) & (gdf_edges['geometory_angle_total'] > 120), 1, 0)

# 開始位置列を追加する
gdf_edges['start_point'] = gdf_edges['geometry'].apply(lambda x: x.coords[0])
# 終了位置列を追加する
gdf_edges['end_point'] = gdf_edges['geometry'].apply(lambda x: x.coords[-1])


# 逆方向のベクトルを持つエッジを削除する
gdf_edges = utils.drop_duplicate_edge(gdf_edges)

# targe_edgesを並び替える
target_edges = gdf_edges[gdf_edges['is_target'] == 1]
target_edges = target_edges.sort_values(['geometory_angle_total'], ascending=[False])

# 地図を表示する
map_osm = ox.plot_graph_folium(graph, edge_width=2)

folium_utils.add_marker(map_osm, latitude_start, longitude_start, "st", "red")
folium_utils.add_marker(map_osm, latitude_end, longitude_end, "ed", "green")

# # is_targetが正のエッジにplotを立てて緯度と経度を表示する
# for index, row in gdf_edges[gdf_edges['is_target'] == 1].iterrows():
#     folium_utils.add_marker(map_osm,
#                             row['geometry'].coords[0][1],
#                             row['geometry'].coords[0][0],
#                             f"{row['geometry'].coords[0][1]}, {row['geometry'].coords[0][0]}, {row['geometory_angle_total']}, {row['length']}",
#                             "red")

# 候補を全て表示
map_osm.add_child(
    folium.features.GeoJson(
        gdf_edges[gdf_edges['is_target'] == 1].to_json(),
        # https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html
        style_function=lambda x: {
            'color': '#0000ff',
            'fill_opacity': 0,
            'weight': 2,
        }
    )
    .add_child(folium.features.GeoJsonPopup(
        fields=['geometory_angle_total', 'length'],
        aliases=['geometory_angle_total', 'length'],
        localize=True
    ))
)

# 候補の上位10件を表示
map_osm.add_child(
    folium.features.GeoJson(
        target_edges.head(5).to_json(),
        # https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html
        style_function=lambda x: {
            'color': '#ff0000',
            'fill_opacity': 0,
            'weight': 5,
        }
    )
    # 緯度と経度を表示する
    .add_child(folium.features.GeoJsonPopup(
        fields=['geometory_angle_total', 'length'],
        aliases=['geometory_angle_total', 'length'],
        localize=True
    ))
)



/tmp/ipykernel_13223/1849061962.py:18: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)
/tmp/ipykernel_13223/1849061962.py:70: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  map_osm = ox.plot_graph_folium(graph, edge_width=2)
